In [ ]:
# Install required packages
!pip3 install folium geopy pandas numpy

In [ ]:
# Import libraries and load data
import pandas as pd
import numpy as np
import folium
from geopy.geocoders import Nominatim
import time
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load the Bexar County $150K+ PPP data
df = pd.read_csv("bexar_county_ppp_150k_plus_loans.csv", encoding='latin-1', on_bad_lines='skip', low_memory=False)  # Update with your CSV filename

print(f"Loaded {len(df):,} PPP loan records ($150K+)")
print(f"Data columns: {list(df.columns)}")
if 'DateApproved' in df.columns:
    print(f"Date range: {df['DateApproved'].min()} to {df['DateApproved'].max()}")
if 'InitialApprovalAmount' in df.columns:
    print(f"Loan amount range: ${df['InitialApprovalAmount'].min():,.2f} to ${df['InitialApprovalAmount'].max():,.2f}")
    print(f"Total loan amount: ${df['InitialApprovalAmount'].sum():,.2f}")

# Display basic info
df.info()
df.head()

In [ ]:
# Data preprocessing - clean and prepare for mapping
# Create a working copy
df_clean = df.copy()

# Remove any duplicate loans (same borrower, address, amount)
duplicate_cols = ['BorrowerName', 'BorrowerAddress', 'InitialApprovalAmount']
available_dup_cols = [col for col in duplicate_cols if col in df_clean.columns]
if available_dup_cols:
    df_clean = df_clean.drop_duplicates(subset=available_dup_cols)
    print(f"After removing duplicates: {len(df_clean):,} records")

# Ensure we have required address columns
required_cols = ['BorrowerName', 'BorrowerAddress', 'BorrowerCity', 'InitialApprovalAmount']
missing_cols = [col for col in required_cols if col not in df_clean.columns]
if missing_cols:
    print(f"Warning: Missing required columns: {missing_cols}")
    print("Available columns:", list(df_clean.columns))

# Clean address data
df_clean['BorrowerAddress'] = df_clean['BorrowerAddress'].astype(str).str.strip()
df_clean['BorrowerCity'] = df_clean['BorrowerCity'].astype(str).str.strip()
df_clean['BorrowerName'] = df_clean['BorrowerName'].astype(str).str.strip()

# Remove records with missing critical info
df_clean = df_clean.dropna(subset=['BorrowerAddress', 'BorrowerCity', 'InitialApprovalAmount'])
df_clean = df_clean[df_clean['BorrowerAddress'] != '']
df_clean = df_clean[df_clean['BorrowerCity'] != '']

print(f"After cleaning: {len(df_clean):,} records ready for geocoding")
print(f"Unique addresses: {df_clean['BorrowerAddress'].nunique():,}")
print(f"Unique borrowers: {df_clean['BorrowerName'].nunique():,}")

# Show loan amount distribution
print(f"\nLoan Amount Distribution:")
print(f"Min: ${df_clean['InitialApprovalAmount'].min():,.2f}")
print(f"Max: ${df_clean['InitialApprovalAmount'].max():,.2f}")
print(f"Mean: ${df_clean['InitialApprovalAmount'].mean():,.2f}")
print(f"Median: ${df_clean['InitialApprovalAmount'].median():,.2f}")

In [ ]:
# Setup local Nominatim geocoder and geocoding function
# Replace 'localhost:8080' with your actual Nominatim server address
LOCAL_NOMINATIM_URL = "http://localhost:8080"  # Adjust this to your server

# Initialize geocoder with local Nominatim server
geolocator = Nominatim(
    user_agent="bexar_150k_ppp_mapping", 
    domain=LOCAL_NOMINATIM_URL.replace('http://', '').replace('https://', ''),
    scheme='http'  # or 'https' if your server uses SSL
)

def normalize_address(address_str):
    """Normalize address string for better geocoding success"""
    if pd.isna(address_str):
        return ""
    
    addr = str(address_str).strip()
    
    # Common abbreviations that cause geocoding failures
    abbreviation_fixes = {
        # Suite variations
        r'\bSte\b\.?': 'Suite',
        r'\bSTE\b\.?': 'Suite', 
        r'\b#': 'Suite ',
        
        # Street type abbreviations
        r'\bBlvd\b\.?': 'Boulevard',
        r'\bBLVD\b\.?': 'Boulevard',
        r'\bAve\b\.?': 'Avenue',
        r'\bAVE\b\.?': 'Avenue',
        r'\bSt\b\.?$': 'Street',
        r'\bST\b\.?$': 'Street',
        r'\bRd\b\.?': 'Road',
        r'\bRD\b\.?': 'Road',
        r'\bDr\b\.?': 'Drive',
        r'\bDR\b\.?': 'Drive',
        r'\bLn\b\.?': 'Lane',
        r'\bLN\b\.?': 'Lane',
        r'\bCt\b\.?': 'Court',
        r'\bCT\b\.?': 'Court',
        r'\bPl\b\.?': 'Place',
        r'\bPL\b\.?': 'Place',
        r'\bPkwy\b\.?': 'Parkway',
        r'\bPKWY\b\.?': 'Parkway',
        r'\bCir\b\.?': 'Circle',
        r'\bCIR\b\.?': 'Circle',
        
        # Direction abbreviations
        r'\bN\b\.?': 'North',
        r'\bS\b\.?': 'South',
        r'\bE\b\.?': 'East',
        r'\bW\b\.?': 'West',
        r'\bNE\b\.?': 'Northeast',
        r'\bNW\b\.?': 'Northwest',
        r'\bSE\b\.?': 'Southeast',
        r'\bSW\b\.?': 'Southwest',
        
        # Common abbreviations
        r'\bBnd\b\.?': 'Bend',
        r'\bLk\b\.?': 'Lake',
        r'\bMt\b\.?': 'Mount',
        r'\bFt\b\.?': 'Fort',
    }
    
    for pattern, replacement in abbreviation_fixes.items():
        addr = re.sub(pattern, replacement, addr, flags=re.IGNORECASE)
    
    # Clean up extra spaces
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

def get_coordinates_local(row, delay=0.1):
    """Enhanced geocoding with multiple fallback strategies"""
    original_address = row.get('BorrowerAddress', '')
    city = row.get('BorrowerCity', '')
    zip_code = str(row.get('BorrowerZip', '')).strip()[:5] if pd.notna(row.get('BorrowerZip')) else ''
    
    # Strategy 1: Full normalized address with ZIP
    normalized_addr = normalize_address(original_address)
    full_address = f"{normalized_addr}, {city}, TX"
    if zip_code and len(zip_code) == 5:
        full_address += f", {zip_code}"
    full_address += ", USA"
    
    strategies = [
        ("Full normalized", full_address),
        ("Without ZIP", f"{normalized_addr}, {city}, TX, USA"),
        ("Without suite", re.sub(r'\bSuite\s+\w+', '', normalized_addr).strip() + f", {city}, TX, USA"),
        ("Street only", re.sub(r'\bSuite.*$', '', normalized_addr).strip() + f", {city}, TX, USA"),
        ("Original format", f"{original_address}, {city}, TX, USA"),
        ("Just street number and name", re.sub(r'\b(Suite|#).*$', '', original_address).strip() + f", {city}, TX, USA")
    ]
    
    # Remove empty strategies
    strategies = [(name, addr) for name, addr in strategies if addr.strip() != f", {city}, TX, USA"]
    
    try:
        for strategy_name, address in strategies:
            if not address.strip() or address.count(',') < 2:
                continue
                
            if strategy_name == "Full normalized":
                print(f"Geocoding: {address[:80]}{'...' if len(address) > 80 else ''}")
            
            try:
                location = geolocator.geocode(address, timeout=10)
                if location:
                    if strategy_name != "Full normalized":
                        print(f"  ✓ Found with {strategy_name}: {location.latitude:.6f}, {location.longitude:.6f}")
                    else:
                        print(f"  ✓ Found: {location.latitude:.6f}, {location.longitude:.6f}")
                    return location.latitude, location.longitude, address
            except Exception as e:
                if strategy_name == "Full normalized":
                    print(f"  ⚠ Error with {strategy_name}: {e}")
                continue
        
        print(f"  ✗ No coordinates found after trying {len(strategies)} strategies")
        return None, None, strategies[0][1] if strategies else full_address
    
    except Exception as e:
        print(f"  ✗ Critical error geocoding: {e}")
        return None, None, full_address
    
    finally:
        if delay > 0:
            time.sleep(delay)

print(f"Geocoder configured for: {LOCAL_NOMINATIM_URL}")
print("Ready to geocode addresses...")

In [ ]:
# Geocode each individual loan recipient
print(f"Starting geocoding of {len(df_clean)} individual loan recipients...")
print("This may take several minutes depending on your server performance.")

# Add columns for coordinates
df_clean['Latitude'] = None
df_clean['Longitude'] = None
df_clean['geocoded_address'] = None

# Track progress
total_loans = len(df_clean)
processed_count = 0
successful_count = 0
start_time = time.time()

for idx, row in df_clean.iterrows():
    # Get coordinates using geocoding function
    lat, lon, geocoded_addr = get_coordinates_local(row, delay=0.1)
    
    # Assign to dataframe
    df_clean.loc[idx, 'Latitude'] = lat
    df_clean.loc[idx, 'Longitude'] = lon
    df_clean.loc[idx, 'geocoded_address'] = geocoded_addr
    
    # Count success
    if lat is not None and lon is not None:
        successful_count += 1
    
    processed_count += 1
    
    # Progress update every 25 addresses
    if processed_count % 25 == 0:
        elapsed = time.time() - start_time
        rate = processed_count / (elapsed / 60) if elapsed > 0 else 0  # addresses per minute
        remaining_time = (total_loans - processed_count) / rate if rate > 0 else 0
        success_rate = successful_count / processed_count * 100 if processed_count > 0 else 0
        
        print(f"Progress: {processed_count}/{total_loans} ({processed_count/total_loans*100:.1f}%) - "
              f"Success: {successful_count} ({success_rate:.1f}%) - "
              f"Rate: {rate:.1f}/min - ETA: {remaining_time:.1f} min")

# Final summary
final_successful = df_clean['Latitude'].notna().sum()
final_failed = len(df_clean) - final_successful

print(f"\n" + "="*60)
print("GEOCODING COMPLETE!")
print("="*60)
print(f"Loan recipients processed: {processed_count:,}")
print(f"Successfully geocoded: {final_successful:,} ({final_successful/len(df_clean)*100:.1f}%)")
print(f"Failed geocodes: {final_failed:,} ({final_failed/len(df_clean)*100:.1f}%)")

# Save results
df_clean.to_csv("bexar_150k_ppp_geocoded.csv", index=False)
print(f"\nGeocoded data saved to: bexar_150k_ppp_geocoded.csv")

In [ ]:
# Create interactive map with individual loan recipients
import folium
from folium.plugins import MarkerCluster

# Filter for successfully geocoded loans
df_map = df_clean.dropna(subset=['Latitude', 'Longitude']).copy()

print(f"Creating map with {len(df_map):,} geocoded loan recipients")

# Create base map centered on San Antonio
ppp_map = folium.Map(
    location=[29.5187, -98.6047], 
    zoom_start=11,
    tiles='OpenStreetMap'
)

# Calculate loan amount ranges for color coding
min_amount = df_map['InitialApprovalAmount'].min()
max_amount = df_map['InitialApprovalAmount'].max()
amount_range = max_amount - min_amount

print(f"Loan amount range: ${min_amount:,.2f} to ${max_amount:,.2f}")

# Define loan amount categories for color coding
def get_marker_color_and_size(loan_amount):
    """Return color and size based on loan amount"""
    if loan_amount >= 2000000:  # $2M+
        return 'darkred', 12
    elif loan_amount >= 1000000:  # $1M+
        return 'red', 10
    elif loan_amount >= 500000:  # $500K+
        return 'orange', 8
    elif loan_amount >= 300000:  # $300K+
        return 'yellow', 7
    elif loan_amount >= 200000:  # $200K+
        return 'lightgreen', 6
    else:  # $150K-200K
        return 'green', 5

# Add individual markers for each loan recipient
for idx, row in df_map.iterrows():
    # Get marker properties based on loan amount
    marker_color, marker_size = get_marker_color_and_size(row['InitialApprovalAmount'])
    
    # Create detailed tooltip content
    tooltip_content = f"""
    <div style="font-family: Arial; width: 300px;">
        <b style="color: #2E86AB; font-size: 14px;">{row['BorrowerName']}</b><br>
        <hr style="margin: 5px 0;">
        <b>📍 Address:</b> {row['BorrowerAddress']}<br>
        <b>🏙️ City:</b> {row['BorrowerCity']}, {row.get('BorrowerState', 'TX')}<br>
        <b>💰 Loan Amount:</b> <span style="color: green; font-weight: bold;">${row['InitialApprovalAmount']:,.2f}</span><br>
    """
    
    # Add optional fields if available
    if 'DateApproved' in row and pd.notna(row['DateApproved']):
        tooltip_content += f"<b>📅 Date Approved:</b> {row['DateApproved']}<br>"
    
    if 'NAICS' in row and pd.notna(row['NAICS']):
        tooltip_content += f"<b>🏢 Industry (NAICS):</b> {row['NAICS']}<br>"
    
    if 'JobsReported' in row and pd.notna(row['JobsReported']):
        tooltip_content += f"<b>👥 Jobs Reported:</b> {int(row['JobsReported'])}<br>"
    
    if 'ForgivenessAmount' in row and pd.notna(row['ForgivenessAmount']):
        forgiveness_pct = (row['ForgivenessAmount'] / row['InitialApprovalAmount']) * 100
        tooltip_content += f"<b>✅ Forgiven:</b> ${row['ForgivenessAmount']:,.2f} ({forgiveness_pct:.1f}%)<br>"
    
    if 'Lender' in row and pd.notna(row['Lender']):
        tooltip_content += f"<b>🏦 Lender:</b> {row['Lender']}<br>"
    
    tooltip_content += "</div>"
    
    # Create circle marker with enhanced styling
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=marker_size,
        color='white',
        weight=2,
        fill=True,
        fillColor=marker_color,
        fillOpacity=0.8,
        popup=folium.Popup(tooltip_content, max_width=350),
        tooltip=f"{row['BorrowerName']}: ${row['InitialApprovalAmount']:,.0f}"
    ).add_to(ppp_map)

print("Markers added successfully!")

In [ ]:
# Add enhanced title
title_html = '''
<div style="position: fixed; 
    top: 10px; left: 50px; width: 500px; height: 90px; 
    background-color: white; border:3px solid #2E86AB; z-index:9999; 
    font-size:16px; font-family: Arial; font-weight: bold;
    text-align: center; padding: 15px; display: flex; align-items: center; justify-content: center;
    box-shadow: 0 4px 8px rgba(0,0,0,0.2); border-radius: 10px;">
    <div>
        <h2 style="margin: 0; color: #2E86AB;">Bexar County PPP Loans</h2>
        <h3 style="margin: 5px 0 0 0; color: #666;">$150,000+ Recipients</h3>
    </div>
</div>
'''
ppp_map.get_root().html.add_child(folium.Element(title_html))

# Add enhanced legend
legend_html = '''
<div style="position: fixed; 
    top: 10px; right: 10px; width: 250px; height: 240px; 
    background-color: white; border:3px solid #2E86AB; z-index:9999; 
    font-size:12px; font-family: Arial;
    padding: 15px; box-shadow: 0 4px 8px rgba(0,0,0,0.2); border-radius: 10px;">
    <h3 style="margin-top: 0; color: #2E86AB; text-align: center;">💰 Loan Amount Ranges</h3>
    <div style="line-height: 1.8;">
        <p style="margin: 5px 0;"><i class="fa fa-circle" style="color:green; font-size:14px;"></i> $150K - $200K</p>
        <p style="margin: 5px 0;"><i class="fa fa-circle" style="color:lightgreen; font-size:16px;"></i> $200K - $300K</p>
        <p style="margin: 5px 0;"><i class="fa fa-circle" style="color:yellow; font-size:18px;"></i> $300K - $500K</p>
        <p style="margin: 5px 0;"><i class="fa fa-circle" style="color:orange; font-size:20px;"></i> $500K - $1M</p>
        <p style="margin: 5px 0;"><i class="fa fa-circle" style="color:red; font-size:22px;"></i> $1M - $2M</p>
        <p style="margin: 5px 0;"><i class="fa fa-circle" style="color:darkred; font-size:24px;"></i> $2M+</p>
    </div>
    <hr style="margin: 10px 0; border-color: #2E86AB;">
    <p style="margin: 5px 0; font-size:11px; color:#666; text-align: center;">
        <b>Circle size</b> = Loan amount<br>
        Click markers for details
    </p>
</div>
'''
ppp_map.get_root().html.add_child(folium.Element(legend_html))

print("Title and legend added!")

In [ ]:
# Add summary stats box
total_amount = df_map['InitialApprovalAmount'].sum()
avg_amount = df_map['InitialApprovalAmount'].mean()

stats_html = f'''
<div style="position: fixed; 
    bottom: 10px; left: 10px; width: 280px; height: 100px; 
    background-color: white; border:3px solid #2E86AB; z-index:9999; 
    font-size:12px; font-family: Arial;
    padding: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.2); border-radius: 10px;">
    <h4 style="margin-top: 0; color: #2E86AB; text-align: center;">📊 Summary Statistics</h4>
    <p style="margin: 3px 0;"><b>Total Recipients:</b> {len(df_map):,}</p>
    <p style="margin: 3px 0;"><b>Total Amount:</b> ${total_amount:,.2f}</p>
    <p style="margin: 3px 0;"><b>Average Loan:</b> ${avg_amount:,.2f}</p>
</div>
'''
ppp_map.get_root().html.add_child(folium.Element(stats_html))

print("Summary statistics box added!")

# Save the interactive map
map_filename = "bexar_county_150k_ppp_loans_map.html"
ppp_map.save(map_filename)

print(f"Interactive map saved as: {map_filename}")
print(f"Map includes {len(df_map):,} loan recipients")
print(f"Total PPP amount mapped: ${df_map['InitialApprovalAmount'].sum():,.2f}")

# Display the map in Jupyter (if running in notebook)
ppp_map